In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from pandas_visual_analysis import VisualAnalysis
import matplotlib.pyplot as plt
import json
from collections import OrderedDict
from datetime import date
import time
import seaborn as sb

In [2]:
data = pd.read_csv('./csv/abusingDetectionTrainDataset.csv')
data.shape

(104399, 44)

In [3]:
data = data.drop(['socialBooleanB', 'accountMetaAmountA', 'charSessionAmountB', 'tradeSessionAmountC', 'tradeSessionAmountE'], axis=1)
data.shape

(104399, 39)

In [4]:
data.columns

Index(['newID', 'char_jobcode', 'char_level', 'logging_timestamp', 'charStatA',
       'charStatB', 'charStatC', 'charStatD', 'charStatE', 'charStatF',
       'charStatG', 'socialAmountA', 'socialBooleanA', 'socialSessionAmountA',
       'activityCumulativeAmountA', 'charSessionAmountA',
       'charMoveSessionAmountA', 'charSessionAmountC', 'socialSessionAmountB',
       'actionSessionAmountA', 'charSessionAmountD',
       'activityCumulativeAmountB', 'actionSessionAmountB',
       'actionSessionAmountC', 'actionSessionAmountD', 'actionSessionAmountE',
       'tradeSessionAmountA', 'tradeSessionAmountB', 'actionSessionAmountF',
       'actionSessionAmountG', 'actionSessionAmountH', 'actionSessionAmountI',
       'actionSessionAmountJ', 'actionSessionAmountK', 'actionSessionAmountL',
       'actionSessionAmountM', 'actionSessionAmountN', 'tradeSessionAmountD',
       'blocked'],
      dtype='object')

# logging_timestamp에서 시간단위로 새로운 컬럼 추가

In [5]:
data_hour = pd.to_datetime(data['logging_timestamp']).dt.strftime('%H').astype(int)
data_hour = data_hour.replace(0, 24)
data['date_hout'] = data_hour
data.shape

(104399, 40)

# tradeSessionAmount A, B, D  ==> tradeSessionAmount 

In [6]:
tradeSessionAmount = data['tradeSessionAmountA'] + data['tradeSessionAmountB'] + data['tradeSessionAmountD']
data = data.drop(['tradeSessionAmountA', 'tradeSessionAmountB', 'tradeSessionAmountD'], axis=1)
data['tradeSessionAmount'] = tradeSessionAmount
data.shape

(104399, 38)

# charSessionAmountC, actionSessionAmountM 제거

In [7]:
data = data.drop(['charSessionAmountC', 'actionSessionAmountM'], axis=1)
data.shape

(104399, 36)

# socialSessionAmountB

In [8]:
socialSessionAmountB = data['socialSessionAmountB']
socialSessionAmountB = [1 if i == 0 else 0 for i in socialSessionAmountB]
data = data.drop(['socialSessionAmountB'], axis=1)
data['socialSessionAmountB'] = socialSessionAmountB


In [9]:
data

,newID,char_jobcode,char_level,logging_timestamp,charStatA,charStatB,charStatC,charStatD,charStatE,charStatF,...,actionSessionAmountH,actionSessionAmountI,actionSessionAmountJ,actionSessionAmountK,actionSessionAmountL,actionSessionAmountN,blocked,date_hout,tradeSessionAmount,socialSessionAmountB
0,0,0,10,2017-6-26 8:59:4.224398,0,0,0,0,0,0,...,0,0,0,0,2,0,0,8,0,1
1,0,0,31,2017-6-26 9:29:6.39562,0,0,0,0,2,0,...,0,0,0,2,146,106,0,9,0,1
2,0,0,40,2017-6-26 9:59:13.175257,0,0,0,0,2,0,...,0,0,0,2,229,172,0,9,0,1
3,0,0,61,2017-6-26 10:29:13.525274,0,0,0,0,6,0,...,1,2,0,2,583,615,0,10,0,1
4,0,0,61,2017-6-27 7:51:56.193386,0,0,0,0,6,0,...,0,1,0,17,1,0,0,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104394,14915,120,47,2017-6-19 16:47:28.622771,1,0,4563,0,4575,0,...,0,4,0,0,89,59,0,16,0,1
104395,14915,132,16,2017-6-19 12:17:5.647646,0,0,4563,0,4563,0,...,0,0,0,0,45,44,0,12,0,1
104396,14915,132,23,2017-6-19 12:47:8.165005,0,0,4563,0,4565,0,...,0,0,0,0,195,153,0,12,0,1
104397,14915,132,27,2017-6-19 13:17:9.600144,0,0,4563,0,4565,0,...,0,1,0,0,193,152,0,13,0,1
